In [7]:
import pandas as pd
import os
import glob
import numpy as np
import warnings

warnings.filterwarnings('ignore', message="A value is trying to be set on a copy of a slice from a DataFrame")
warnings.filterwarnings('ignore', category=UserWarning) 

pd.set_option('future.no_silent_downcasting', True)

In [8]:

INPUT_DIR = './rawdata/oews/'
OUTPUT_DIR = './data/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"input dir: {INPUT_DIR}")
print(f"output dir: {OUTPUT_DIR}")

input dir: ./rawdata/oews/
output dir: ./data/


In [9]:
wage_cols = [
    'TOT_EMP', 'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 
    'H_MEAN', 'A_MEAN', 'MEAN_PRSE', 
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90',
    'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90'
]
def process_oews_data(file_path):
    """read single OEWS files，extract year，and pre-cleaning dataset"""
    try:
        #extract year from file name
        year = int(os.path.basename(file_path).split('_')[-1].replace('.xlsx', ''))
        df = pd.read_excel(file_path)
        #drop all na rows
        df.dropna(how='all', inplace=True)
    
        # cleaning: sussitute all non-numeric data-> NaN
        # BLS dataset contains：'#', '**', '*' in rows as values for privacy/insufficient data etc
        symbols_to_replace = ['#', '**', '*']
        for col in df.columns:
            if col in wage_cols:
                df[col] = df[col].replace(symbols_to_replace, np.nan)
        for col in wage_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        boolean_cols = ['ANNUAL', 'HOURLY']
        for col in boolean_cols:
            if col in df.columns:
                df[col] = df[col].astype(str).str.lower().map({'true': True, 'false': False, '1': True, '0': False, '': False})
        if 'AREA' in df.columns:
            df['AREA'] = pd.to_numeric(df['AREA'], errors='coerce').astype('Int64')
            
        # add new column:year
        df['YEAR'] = year
        selected_cols = [
            'YEAR', 'AREA', 'AREA_TITLE', 'PRIM_STATE', 
            'NAICS', 'NAICS_TITLE', 'OCC_CODE', 'OCC_TITLE', 
            'TOT_EMP', 'A_MEAN', 'A_MEDIAN', 'A_PCT10', 'A_PCT90', 
            'ANNUAL', 'HOURLY' 
        ]
        final_cols = [col for col in selected_cols if col in df.columns]
        return df[final_cols]
    except Exception as e:
        print(f"error when processing file: {file_path} : {e}")
        return None


In [10]:
        
# batch reading and combine
all_files = glob.glob(os.path.join(INPUT_DIR, 'all_data_M_20*.xlsx'))
print(f"Found {len(all_files)} files need to be processed...")

all_data = [process_oews_data(f) for f in all_files]
all_data = [df for df in all_data if df is not None]
if all_data:
    df_combined = pd.concat(all_data, ignore_index=True)
    print(f"successfully combined {len(all_data)} files. Total records: {len(df_combined)}")
else:
    print("!!!!error:check file path")
    df_combined = pd.DataFrame() 


Found 9 files need to be processed...
successfully combined 9 files. Total records: 3727879


In [11]:
df_filtered = df_combined.copy()

# filtering
# filter out rows with'00-0000': 'All Occupations'
df_filtered = df_filtered[df_filtered['OCC_CODE'] != '00-0000']
    
# filter out rows'000000': 'Cross-industry' or 'All Industry'
df_filtered = df_filtered[df_filtered['NAICS'] != '000000']

# cleaning rigion data :filtering rows where PRIM_STATE = na
df_filtered.dropna(subset=['PRIM_STATE'], inplace=True)

# filter out when key column has na vals (TOT_EMP and A_MEDIAN)
df_filtered.dropna(subset=['TOT_EMP', 'A_MEDIAN'], inplace=True)
    

for col in ['TOT_EMP', 'A_MEAN', 'A_MEDIAN']:
    if col in df_filtered.columns:
        df_filtered[col] = df_filtered[col].astype(float)
    
print(f"total records after cleaning: {len(df_filtered)}")
    

print("\n sample data:")
print(df_filtered.head())
print(f"\n info:")
print(df_filtered.info())



total records after cleaning: 639953

 sample data:
      YEAR  AREA AREA_TITLE PRIM_STATE   NAICS  \
1404  2021    99       U.S.         US  000001   
1405  2021    99       U.S.         US  000001   
1408  2021    99       U.S.         US  000001   
1409  2021    99       U.S.         US  000001   
1410  2021    99       U.S.         US  000001   

                                 NAICS_TITLE OCC_CODE  \
1404  Cross-industry, private ownership only  11-0000   
1405  Cross-industry, private ownership only  11-1000   
1408  Cross-industry, private ownership only  11-1020   
1409  Cross-industry, private ownership only  11-1021   
1410  Cross-industry, private ownership only  11-2000   

                                              OCC_TITLE    TOT_EMP    A_MEAN  \
1404                             Management Occupations  7754600.0  125450.0   
1405                                     Top Executives  3006050.0  121360.0   
1408                    General and Operations Managers  2839990